<a href="https://colab.research.google.com/github/ian-byrne/MADSmilestone2/blob/main/multimodel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multiple Model Option

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/ian-byrne/MADSmilestone2.git
# Change directory into cloned repo
%cd MADSmilestone2

fatal: destination path 'MADSmilestone2' already exists and is not an empty directory.
/content/MADSmilestone2


In [3]:
!git pull

Already up to date.


In [4]:
!pip install boto3

In [5]:
!ls

Data			 Labeling	       README.md
ian_testingground.ipynb  Loading	       Supervised
ImagePlayground		 Model_Datasets.ipynb  Supervised_CNN.ipynb


In [6]:
# General Libraries
import pandas as pd
import numpy as np
import ast
# Custom Libraries
import Loading.load_data as ld

# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# To Evaluate model
from tqdm import tqdm
# import torchmetrics
# from torchmetrics import ConfusionMatrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# To visualize model
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from skimage.io import imread

# To split the data
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
from botocore.exceptions import ClientError
from torch.nn.modules.activation import ReLU
# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [8]:
from torch.utils.data import Dataset, DataLoader
# import from module later
class ResizedClocks(Dataset):
    #Resized clock drawing dataset

    def __init__(self, round, round_labels):
        
       # Args:
           # round (int): Round to grab images from. 
           # values (list of tuples): Corresponding values for the round.
        
        self.round = round
        self.vals = round_labels
        self.client = boto3.client('s3', 
                                    aws_access_key_id=pubkey, 
                                    aws_secret_access_key=seckey)
    
    def __len__(self):
        return len(self.vals)

    def __getitem__(self, idx):
        spid = self.vals[idx][0]
        label = self.vals[idx][1]
        bucket = "clockimages" #"test-bucket-clockids-aicrowd"
        obj_name = f"NHATS_R{self.round}_ClockDrawings/{spid}.tif" #f"{self.round}_{spid}.tif"
        #filename = str(spid)+".tif"
        temp = tempfile.NamedTemporaryFile()

        try:
          client.download_file(bucket, obj_name, temp.name)

          im = Image.open(temp.name)

          gray = im.convert('1')
          resized = gray.resize((160, 207)) 
          im_arr = np.array(resized).astype(int)

          sample = {'image': im_arr, 'label': label}
        
          temp.close()

          return sample
          
        except botocore.exceptions.ClientError as e:
          #logging.error(e)
          pass

In [9]:
file = open("Data/Dictionaries/score_dicts/tr_scor_dict_bal.txt", "r")

contents = file.read()
im_scores = ast.literal_eval(contents)

In [10]:
#num_epochs = 20
batch_size = 8
learning_rate = .001
kernel_size = 3
stride = 1
padding = 1 #2*floor(3/2)

accuracy_stats = {
    'train': [],
    'val': []
}

loss_stats = {
    'train': [],
    'val': []
}

In [11]:
path = "/content/gdrive/MyDrive/numpy_files/Score_data/"
training_data, y_train_tensor = ld.load_np_files(path+"train_score_im.npy", path+"train_score_labels.npy")

(23621, 1, 368, 284)


In [12]:
validation_data, y_val_tensor = ld.load_np_files(path+"val_score_im.npy", path+"val_score_labels.npy")
test_data, y_test_tensor = ld.load_np_files(path+"tst_score_im.npy", path+"tst_score_labels.npy")

(2544, 1, 368, 284)
(2541, 1, 368, 284)


In [13]:
y_train_tensor = y_train_tensor.to(torch.long)
y_test_tensor = y_test_tensor.to(torch.long)
y_test_tensor = y_test_tensor.to(torch.long)

In [14]:
y_train_tensor.dtype

torch.int64

In [15]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size = batch_size, shuffle = False) 
validate_loader = torch.utils.data.DataLoader(validation_data, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False)
#Labels 
classes = (0, 1, 2, 3, 4, 5)

In [16]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    # without considering batch size: Input shape : (None,368, 284, 1) , parameters: (3*3*1*16+16) = 160
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, # one input channel gray scale, 16 filters out
                            kernel_size = 3, stride = 1, padding = 1) #Out:(None,386, 284, 16)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, 
                          kernel_size = 3, stride = 1, padding = 1) #params: (3*3*16*32+32) = 4640                        
    self.pool1 = nn.MaxPool2d(2, 2) #Out: (None, 184, 142, 32)
    self.bn1 = nn.BatchNorm2d(32)

    self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, 
                          kernel_size = 3, stride = 1, padding = 1) #params: (3*3*16*32+32) = 4640    
    self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, 
                          kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248                     
    self.pool2 = nn.MaxPool2d(2, 2) #Output shape = (None, 92, 71, 64)
    self.bn2 = nn.BatchNorm2d(64) 

    self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 128, 
                          kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248 
    self.conv6 = nn.Conv2d(in_channels = 128, out_channels = 128, 
                          kernel_size = 3, stride = 1, padding = 1) # params: (3*3*32*32+32) = 9248
    self.pool3 = nn.MaxPool2d(2, 2) #Output shape = (None, 46, 35, 128)
    self.bn3 = nn.BatchNorm2d(128)
    
    # Fully connected layer
    self.fc1 = nn.Linear(128*46*35,6)
    #self.fc2 = nn.Linear(120, 60)
    #self.fc3 = nn.Linear(60, 30)
    #self.fc4 = nn.Linear(30, 3) # left with 3 for the three classes 

  def forward(self, x):
    x = self.bn1(self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))))
    x = self.bn2(self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))))
    x = self.bn3(self.pool3(F.relu(self.conv6(F.relu(self.conv5(x))))))
    x = x.view(x.size(0),128*46*35)
    x = self.fc1(x)


    return x 

In [23]:
def accuracy(y_pred, y_test):
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
  
  #y_preds = y_pred.argmax(dim=1)

  correct_pred = (y_pred_prob == y_test).float()
  #print("correct sum: ", correct_pred.sum())
  #print('correct total length: ', len(correct_pred))
  #print(correct_pred)
  acc = correct_pred.sum() / len(correct_pred)
  
  #acc = correct_pred.sum().float() / float( y_test.size(0) )

  acc = torch.round(acc * 100)

  return acc

In [24]:
# Create model object 
model = ConvNet()
if torch.cuda.is_available():
    model = model.to(float).cuda()
    print('Model training on GPU')
else:
    print("CUDA is not available. Training on CPU...")

#for param in model.parameters():
  #print(str(param.data.numpy().shape)+'\n')
  #print("weights fc1: ", model.fc1.weight)

# Loss function
criterion = nn.CrossEntropyLoss(reduction="mean")

# Optimizer (can use SGD or ADAM)
#optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9) #or ADAM/ momentum

print(model)

Model training on GPU
ConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track

In [27]:
def train_val_model(epochs):
  for epoch in range(1, epochs + 1):

    # TRAINING ********************************
    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode (recommended)
    model.train()

    
    # Double check
    tr_run_loss=0
    tr_correct=0
    tr_total=0
    train_accu = []
    train_losses = []
    
    print('\nEpoch$ : %d'%epoch)
    for x_train_batch, y_train_batch in tqdm(train_loader):
      x_train_batch = x_train_batch.to(float).to(device) # for GPU support
      y_train_batch = y_train_batch.to(torch.long).to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()

      # Forward pass through NN
      y_train_pred = model(x_train_batch)#.to(float))
      train_loss = criterion(y_train_pred, y_train_batch)
      train_acc = accuracy(y_train_pred, y_train_batch)

      # Backward pass, updating weights
      train_loss.backward()
      optimizer.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()

      # Double check scores
      tr_run_loss += train_loss.item()
     
      _, predicted = y_train_pred.max(1)
      tr_total += y_train_batch.size(0)
      tr_correct += predicted.eq(y_train_batch).sum().item()
       
    tr_loss = tr_run_loss/len(train_loader)
    accu = 100.*tr_correct/tr_total
   
    train_accu.append(accu)
    train_losses.append(tr_loss)
    print('Train Loss: %.3f | Train Accuracy: %.3f'%(tr_loss,accu))
    # VALIDATION****************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      # Double check
      val_run_loss=0
      val_correct=0
      val_total=0
      val_accu = []
      val_losses = []


      model.eval()
      for x_val_batch, y_val_batch in validate_loader:
      
        x_val_batch =  x_val_batch.to(float).to(device)
        y_val_batch = y_val_batch.to(torch.long).to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

        # Double check
        
        val_run_loss += val_loss.item()
     
        _, predictedv = y_val_pred.max(1)
        val_total += y_train_batch.size(0)
        val_correct += predictedv.eq(y_val_batch).sum().item()
       
      vl_loss = val_run_loss/len(validate_loader)
      accuv = 100.*val_correct/val_total
   
      val_accu.append(accuv)
      val_losses.append(vl_loss)
      print('Validation Loss: %.3f | Validation Accuracy: %.3f'%(vl_loss,accuv))


    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

In [28]:
train_val_model(20)


Epoch$ : 1


100%|██████████| 2953/2953 [04:05<00:00, 12.02it/s]


Train Loss: 82025023494401968.000 | Train Accuracy: 90.356
Validation Loss: 41388445545193805054077829120.000 | Validation Accuracy: 38.616
Epoch 001: Train Loss: 82025023494401968.00000 | Val Loss: 41388445545193805054077829120.00000
Train Acc: 90.357 | Val Acc: 24.101

Epoch$ : 2


100%|██████████| 2953/2953 [04:05<00:00, 12.05it/s]


Train Loss: 7853103009110312484385849344.000 | Train Accuracy: 91.067
Validation Loss: 8926887819846007304380326346752.000 | Validation Accuracy: 38.616
Epoch 002: Train Loss: 7853103009110312484385849344.00000 | Val Loss: 8926887819846007304380326346752.00000
Train Acc: 91.068 | Val Acc: 24.101

Epoch$ : 3


...

Train Loss: 1560312281610386833906722550776533614592.000 | Train Accuracy: 88.561
Validation Loss: 667630177403319877759243114909198578614272.000 | Validation Accuracy: 38.616
Epoch 003: Train Loss: 1560312281610386833906722550776533614592.00000 | Val Loss: 667630177403319877759243114909198578614272.00000
Train Acc: 88.562 | Val Acc: 24.101

Epoch$ : 4


100%|██████████| 2953/2953 [04:05<00:00, 12.03it/s]


Train Loss: 125898058266615370724531390959759868299056661921792.000 | Train Accuracy: 90.729
Validation Loss: 138930532496926959838073540777244644256823133308190720.000 | Validation Accuracy: 38.616
Epoch 004: Train Loss: 125898058266615370724531390959759868299056661921792.00000 | Val Loss: 138930532496926959838073540777244644256823133308190720.00000
Train Acc: 90.730 | Val Acc: 24.101

Epoch$ : 5


 16%|█▌        | 465/2953 [00:38<03:27, 12.02it/s]


KeyboardInterrupt: ignored